In [ ]:
!pip install googledrivedownloader;
from google_drive_downloader import GoogleDriveDownloader as gdd

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def tokenize(sentences):

    tokenized_senteces = list()
    for s, sent in enumerate(sentences):

        remove_punctuation = RegexpTokenizer(r"\w+")
        tokenized_sent = remove_punctuation.tokenize(sent)

        for i, t in enumerate(tokenized_sent):
            if t.isnumeric():
                tokenized_sent[i] = "<num>"

        tokenized_senteces.append(tokenized_sent)

    return tokenized_senteces


def token_filter(tokenized_sentences, thresh=5):
    
    words = list()
    for t_sent in tokenized_sentences:
        for word in t_sent:
            words.append(word)

    vocab = Vocabulary(words, unk_cutoff=thresh)
    print("Length of vocab: {}".format(len(vocab)))
    filtered_sentences = list()

    for t, tokenized_sent in enumerate(tokenized_sentences):
        filtered_sent = list()
        for word in tokenized_sent:
            if vocab.lookup(word) == '<UNK>':
                filtered_sent.append('<unk>')
            else: 
                filtered_sent.append(word)
        filtered_sentences.append(filtered_sent)

    return filtered_sentences

In [ ]:
!pip install fasttext
import fasttext
import pandas as pd

def Skipgram(filtered_sentences, ws=3, dim=50):
    model = fasttext.train_unsupervised(filtered_sentences, model='skipgram', ws=ws, dim=dim, neg=5)
    vocab_dict = {} 
    vocab_dict = {word:idx for (idx, word) in enumerate(model.get_words())}
    return model




     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093634 sha256=1ea9742ef714bfd9f6085cfa2a86a609850e73b76276477e3a7b7d76bda8add6
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
## Download pre-processed data
gdd.download_file_from_google_drive(file_id='1PUiB33hgTsefasb3D2t920Gu2mINLd4C', dest_path='./preprocessed_data/preprocessed_data.zip', unzip=True)
!rm ./preprocessed_data/preprocessed_data.zip



trn = pd.read_csv('./preprocessed_data/trn_title.csv', delimiter = ',', names=['title','label'])
tst = pd.read_csv('./preprocessed_data/tst_title.csv', delimiter = ',', names=['title','label'])

In [ ]:
print(trn.shape)
print(tst.shape)

(31427, 2)
(4492, 2)


In [ ]:
!pip install nltk==3.6

import math
import pickle
import urllib.request
from tqdm import tqdm
from os.path import isfile

import nltk
nltk.download("punkt")
from nltk import RegexpTokenizer
from nltk.lm import Vocabulary

trn_sentences = tokenize(list(trn.title))
trn_filtered_sentences = token_filter(trn_sentences,5)

tst_sentences = tokenize(list(tst.title))
tst_filtered_sentences = token_filter(tst_sentences,5)

     |████████████████████████████████| 1.5MB 4.7MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Length of vocab: 9495
Length of vocab: 2169


In [ ]:
print(trn_sentences[0])

['Texas', 'Moving', 'To', 'Ban', 'Jews', 'Muslims', 'and', 'Gays', 'from', 'Adopting']


In [ ]:
with open('../preprocessed_embeddings/sg_train.txt', 'w') as f:
    for title in trn_filtered_sentences:
        for w in title: 
          f.write("%s " % w)
        f.write("\n")

with open('../preprocessed_embeddings/sg_test.txt', 'w') as f:
    for title in tst_filtered_sentences:
        for w in title: 
          f.write("%s " % w)
        f.write("\n")






model_sg = Skipgram('./preprocessed_data/sg_train.txt')
model_sg.save_model("model_sg.bin")


In [ ]:
f1 = open("./preprocessed_data/sg_train.txt", "r")
f2 = open("./preprocessed_data/sg_test.txt", "r")

trn_embeddings = []
tst_embeddings = [] 

for line in f1:
    line_stripped = line.strip()  
    trn_embeddings.append(model_sg.get_sentence_vector(line_stripped))

f1.close()


for line in f2:
    line_stripped = line.strip()
    tst_embeddings.append(model_sg.get_sentence_vector(line_stripped))
